## Question 1: **Answer**

In [1]:
import pandas as pd
import sqlite3

In [ ]:
def add_data_to_database(input_data):
    try:
        # Creating a DataFrame
        df = pd.read_csv(input_data)
        # Connecting to database
        engine = sqlite3.connect("social_media_platforms.db")
        df.to_sql("social_media", con=engine, index = False)
        engine.close()
    except Exception as e:
            print(f'Error: {e}')

# Loading data
input_data = "/content/social_media_platforms.csv"
add_data_to_database(input_data)

In [3]:
# Loading Magic extension
%load_ext sql

In [4]:
# Connecting to the database
%sql sqlite:///social_media_platforms.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM social_media
LIMIT 5;

 * sqlite:///social_media_platforms.db
Done.


Date,Facebook,Pinterest,Twitter,StumbleUpon,YouTube,Instagram,Tumblr,reddit,VKontakte,LinkedIn,Google+,Digg,MySpace,Fark,NowPublic,iWiW,news.ycombinator.com,Delicious,orkut,Odnoklassniki,Vimeo,Other
2009-04,20.16,0.0,6.86,36.79,0.0,0.0,0.0,8.98,0.0,0.0,0.0,6.7,14.81,0.22,0.04,0.29,0.08,0.49,1.75,0.0,0.0,2.83
2009-05,24.3,0.0,9.95,33.78,0.0,0.0,0.0,7.62,0.0,0.28,0.0,7.34,8.95,0.44,0.05,2.1,0.12,0.57,1.77,0.0,0.0,2.72
2009-06,26.48,0.0,10.56,29.65,0.0,0.0,0.0,12.13,0.0,0.49,0.0,5.9,7.66,0.66,0.05,2.35,0.11,0.58,1.46,0.0,0.0,1.93
2009-07,29.1,0.0,10.35,33.55,0.0,0.0,0.0,9.52,0.0,0.41,0.0,5.36,5.49,0.22,0.03,2.21,0.14,0.55,1.24,0.0,0.0,1.84
2009-08,34.25,0.0,11.15,29.01,0.0,0.0,0.0,9.37,0.0,0.45,0.0,4.56,4.69,0.4,0.08,2.27,0.08,0.52,1.27,0.0,0.0,1.9


## Question 2: **Answer**

In [7]:
%%sql
-- Fetch previous year december value
WITH december_percentages AS (
    SELECT
    Date,
    Facebook,
    -- Facebook percentage from the previous December
    LAG(Facebook) OVER (ORDER BY Date)
    AS previous_year
FROM social_media
-- Filter for December records only
WHERE Date LIKE '%-12')

-- Calculate the absolute year-over-year change for December 2010
SELECT
    Date,
    Facebook AS facebook_percentage_share,
    ABS(ROUND((Facebook - previous_year), 2))
    AS percentage_change,
    (CASE
        WHEN (Facebook - previous_year) > 0 Then 'Growth'
        ELSE 'Decline'
    END) AS comment
FROM december_percentages
WHERE Date LIKE '2010-12%'

 * sqlite:///social_media_platforms.db
Done.


Date,facebook_percentage_share,percentage_change,comment
2010-12,60.88,13.51,Decline


## Question 3: **Answer**

In [8]:
%%sql
-- Filter data for 2012 and calculate the previous month's values using LAG
WITH month_over_month_change AS (
    SELECT
        Date,
        Twitter,
        YouTube,
        LAG(Twitter) OVER (ORDER BY Date) AS prev_month_twitter,
        LAG(YouTube) OVER (ORDER BY Date) AS prev_month_youtube
   FROM social_media
    -- Filter for 2012
   WHERE Date LIKE '2012-%')

--Calculate percentage change for March 2012
SELECT
    Date,
    Twitter AS Twitter_percentage_share,
    ROUND((Twitter - prev_month_twitter) / prev_month_twitter * 100, 2)
    AS twitter_mom_change,
    YouTube AS Youtube_percentage_share,
    ROUND((YouTube - prev_month_youtube) / prev_month_youtube * 100, 2)
    AS youtube_mom_change
FROM month_over_month_change
WHERE Date LIKE '2012-03%';

 * sqlite:///social_media_platforms.db
Done.


Date,Twitter_percentage_share,twitter_mom_change,Youtube_percentage_share,youtube_mom_change
2012-03,5.33,-11.75,10.97,103.53


## Question 4: **Answer**

In [9]:
%%sql
WITH last_three_months AS (
    SELECT
        Date AS date_values,
        YouTube,
        -- YouTube percentage from the previous month
        LAG(YouTube) OVER (ORDER BY Date) AS prev_youtube
    FROM social_media
    -- Filter for the last 3 months of 2015
    WHERE Date LIKE '2015-10%' OR Date LIKE '2015-11%'
               OR Date LIKE '2015-12%'
)

-- Calculating the month-over-month percentage change
SELECT
    date_values,
    ROUND(YouTube, 2) AS percentage_share,
    --Using COALESCE to return 0 if change is None
    COALESCE(ROUND((YouTube - prev_youtube) / prev_youtube * 100, 2), 0)
    AS youtube_percentage_change
FROM last_three_months;

 * sqlite:///social_media_platforms.db
Done.


date_values,percentage_share,youtube_percentage_change
2015-10,0.12,0
2015-11,0.17,41.67
2015-12,0.23,35.29


## Question 5: **answer**

In [10]:
%%sql
-- Calculate the year-over-year growth for each platform
WITH year_over_year AS (
    SELECT
        Date,
        Reddit,
        LinkedIn,
        VKontakte,
        LAG(Reddit) OVER (ORDER BY Date) AS prev_reddit,
        LAG(LinkedIn) OVER (ORDER BY Date) AS prev_linkedin,
        LAG(VKontakte) OVER (ORDER BY Date) AS prev_vk
    FROM social_media
    WHERE Date LIKE '%-12'
)
-- Calculate the growth or return 2009 percentage for each platform
SELECT
    CASE
        WHEN Reddit > prev_reddit THEN ROUND((Reddit - prev_reddit) / prev_reddit * 100, 2)
        ELSE ROUND(prev_reddit, 2) -- No growth, return 2009 percentage
    END AS Reddit_growth_or_share,
    CASE
        WHEN LinkedIn > prev_linkedin THEN ROUND((LinkedIn - prev_linkedin) / prev_linkedin * 100, 2)
        ELSE ROUND(prev_linkedin, 2) -- No growth, return 2009 percentage
    END AS LinkedIn_growth_or_share,
    CASE
        WHEN VKontakte > prev_vk THEN ROUND((VKontakte - prev_vk) / prev_vk * 100, 2)
        ELSE ROUND(prev_vk, 2) -- No growth, return 2009 percentage
    END AS VKontakte_growth_or_share
FROM year_over_year
WHERE Date = '2010-12';

 * sqlite:///social_media_platforms.db
Done.


Reddit_growth_or_share,LinkedIn_growth_or_share,VKontakte_growth_or_share
26.69,57.14,0.0
